# Text Treatment

In [171]:
import pandas as pd
from datetime import datetime
import unidecode
import re

#Création fonctions de netoyage de texte
import spacy #https://spacy.io/usage  ----> POUR INSTALLER
#https://www.stat4decision.com/fr/traitement-langage-naturel-francais-tal-nlp/

from nltk.stem.snowball import SnowballStemmer


In [83]:
#importer les données texte

pd.set_option('display.max_rows', 500)
df_X = pd.read_csv(r'C:\Users\Edgar\Documents\Rakuten\X_train_update.csv',index_col =0)
df_X.head(20)

,designation,description,productid,imageid
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786
5,Afrique Contemporaine N° 212 Hiver 2004 - Doss...,NaN,5862738,393356830
6,Christof E: Bildungsprozessen Auf Der Spur,NaN,91920807,907794536
7,Conquérant Sept Cahier Couverture Polypro 240 ...,CONQUERANT CLASSIQUE Cahier 240 x 320 mm seyès...,344240059,999581347
8,Puzzle Scooby-Doo Avec Poster 2x35 Pieces,NaN,4239126071,1325918866
9,Tente Pliante V3s5-Pro Pvc Blanc - 3 X 4m50 - ...,Tente pliante V3S5 Pro PVC 500 gr/m² - 3 x 4m5...,3793572222,1245644185


In [76]:
#Merger les colones texte en une seule
X = df_X.designation.astype(str) + ' ' + df_X.description.astype(str)

#Numero de lignes du texte
size_X = X.size
print('Num lignes totale:', size_X)

X.head(20)

Num lignes totale: 84916


0     Olivia: Personalisiertes Notizbuch / 150 Seite...
1     Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...
2     Grand Stylet Ergonomique Bleu Gamepad Nintendo...
3     Peluche Donald - Europe - Disneyland 2000 (Mar...
4     La Guerre Des Tuques Luc a des id&eacute;es de...
5     Afrique Contemporaine N° 212 Hiver 2004 - Doss...
6        Christof E: Bildungsprozessen Auf Der Spur nan
7     Conquérant Sept Cahier Couverture Polypro 240 ...
8         Puzzle Scooby-Doo Avec Poster 2x35 Pieces nan
9     Tente Pliante V3s5-Pro Pvc Blanc - 3 X 4m50 - ...
10    Eames Inspired Sxw Chair - Pink - Black The ti...
11    Fauteuil Chesterfield Brenton 100% Cuir De Buf...
12    Peaceable Kingdom Wheres Bear? The Hide And Fi...
13    Paire De Voilages Imprimés Fantaisie Paire de ...
14    Matelas Mémoire De Forme 180x200 X 20 Cm Très ...
15    Zenith Pince Agrafeuse 591 N°10 Coloris Noir P...
16    Walter Scott Oeuvres Complètes Tomes 3456 10 E...
17              Mod Podge Dishwasher Safe Gloss 

In [203]:
def tokeniser(sentence,language): #Tokenisation par phrases
    
    langue = {'french':'fr_core_news_sm',
          'german':'de_core_news_sm',
          'english':'en_core_web_sm'}
    
    nlp = spacy.load(langue[language])
    # Tokeniser la phrase
    tokens = nlp(sentence)
    # Retourner le texte coupé par mots(token)
    return [token for token in tokens]

def language_(language):
    langue = {'fr':'french',
             'de':'german',
             'en':'english'}
    return langue[language]

def stop_words(sentence,language): #Enleve stop words per phrase
    clean_words = ''
    stop_words =stopwords.words(language)
    tokens = tokeniser(sentence,language)
    for token in tokens:
        if token.text not in stop_words:
            clean_words += token.text + ' '
    return clean_words

def number_unique_words(text_series): #count unique words in Series
    words = []
    for sentence in text_series.str.split():
        for word in sentence:
            words.append(word)
            
    return len(list(set(words)))

def remove_accents(sentece,language): #removes accents
    #language is not used in this function, but needs to be added on the definition as the rest
    try:
        text = unidecode.unidecode(sentece)
    except:
        pass
    return text

def steem(sentence,language): #Steeming
    steemed = ''
    tokens = tokeniser(sentence,language)
    stemmer = SnowballStemmer(language=language)
    
    for token in tokens:
        steemed += stemmer.stem(token.text) +' '
    return steemed

def lemma(sentence,language): #Lemmatizer
    lemma = ''
    tokens = tokeniser(sentence,language)
    
    for token in tokens:
        lemma += token.lemma_ + ' '
    return lemma

def no_num(sentence,language): #delete numbers
    #language is not used in this function, but needs to be added on the definition as the rest
    return ''.join([word for word in sentence if not word.isdigit()])

def no_special(sentence,language): #delete special characters
    #language is not used in this function, but needs to be added on the definition as the rest
    return re.sub(r"[^a-zA-Z0-9]+", ' ', sentence)

In [209]:
#Le traitement de text est très long et des fois on lance le code et après plusieurs heures de tourner on ne sait pas si le code a bientôt fini ou pas
# Ce code divise le dataframe en batchs afin de voir l'avancement du traitement à chaque fin de batch

def treat_text(text_series,text_treat_funct,language,batch,old_treatments=''):
    
    treatments = '_' + text_treat_funct.__name__ + '_' + language
    language = language_(language)
    
    print('************************************************')
    print('')
    print(text_treat_funct.__name__.upper(),language.upper())
    print("Treatment Start Time =", datetime.now().strftime("%H:%M:%S"))
    print("Number of unique words at start =", number_unique_words(text_series))
    #print('Type of function:', text_treat_funct.__name__)
    #print('Treatment language choosen:', language)
    print('Number of batch:', batch)
    
    size = text_series.size
    treated_text = pd.Series()
    text = text_series.str.lower()
    
    #treat batches
    for i in range(batch):
        treated_batch = text[int(size/batch) * i:int(size/batch) * (i + 1)].apply(lambda cell: text_treat_funct(cell,language))
        treated_text = pd.concat([treated_text,treated_batch])
        
        print('Treatment Batch', i, 'from', batch ,'; Time =', datetime.now().strftime("%H:%M:%S"))
    
    #tret last batch
    treated_batch = text[int(size/batch) * (i + 1):size].apply(lambda cell: text_treat_funct(cell,language))
    treated_text = pd.concat([treated_text,treated_batch])
    
    treated_text.to_csv('X_train' + old_treatments + treatments + '.csv') 
    
    print("Number of unique words at end =", number_unique_words(treated_text))
    print('')
    
    return treated_text, old_treatments + treatments

#X_test = X[:23]
#treat_text(X_test,no_special,'french',7)

In [ ]:
#on va creer differents fichiers X_train avec des differents traitements. Ça nous permettra de comprarer les résultats des modèlisations en fonction du traitement
function_list = [stop_words, remove_accents, steem, lemma, no_num, no_special]
langues = ['french','english','german']


batch = 20


X_treated, treatments = treat_text(X,lemma,'fr',batch)
X_treated, treatments = treat_text(X_treated,lemma,'en',batch,treatments)
X_treated, treatments = treat_text(X_treated,lemma,'de',batch,treatments)
X_treated, treatments = treat_text(X_test,stop_words,'fr',batch)
X_treated, treatments = treat_text(X_treated,stop_words,'en',batch,treatments)
X_treated, treatments = treat_text(X_treated,stop_words,'de',batch,treatments)
X_treated, treatments = treat_text(X_test,remove_accents,'fr',batch)
X_treated, treatments = treat_text(X_treated,no_num,'fr',batch,treatments)
X_treated, treatments = treat_text(X_treated,no_special,'fr',batch,treatments)

X_treated, treatments = treat_text(X_treated,steem,'fr',batch,treatments)
X_treated, treatments = treat_text(X_treated,steem,'en',batch,treatments)
X_treated, treatments = treat_text(X_treated,steem,'de',batch,treatments)

************************************************

LEMMA FRENCH
Treatment Start Time = 09:00:55
Number of unique words at start = 407222
Number of batch: 20


<ipython-input-209-f0dd036dfd0b>:19: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  treated_text = pd.Series()


Treatment Batch 0 from 20 ; Time = 10:02:46
Treatment Batch 1 from 20 ; Time = 11:04:47
Treatment Batch 2 from 20 ; Time = 12:06:51
Treatment Batch 3 from 20 ; Time = 13:08:56
Treatment Batch 4 from 20 ; Time = 14:11:09
Treatment Batch 5 from 20 ; Time = 15:13:28
Treatment Batch 6 from 20 ; Time = 16:15:34
Treatment Batch 7 from 20 ; Time = 17:17:51
Treatment Batch 8 from 20 ; Time = 18:20:13
Treatment Batch 9 from 20 ; Time = 19:22:40
Treatment Batch 10 from 20 ; Time = 20:25:16
Treatment Batch 11 from 20 ; Time = 21:28:05
Treatment Batch 12 from 20 ; Time = 22:31:01
Treatment Batch 13 from 20 ; Time = 23:33:44
